**GROUP PROJECT PROPOSAL**


# **USING AGE, HEARTRATE, AND CHOLESTROL TO CLASSIFY HEART PATIENTS IN CLEVELAND**
#### ARYAN JAIN, KATIE LIANG, MADILYN PORTAS, VIBHAV

### The term cadio vascular disease 